[Dataset](https://www.kaggle.com/arashnic/building-sites-power-consumption-dataset)

In [2]:
#!kaggle datasets download -d arashnic/building-sites-power-consumption-dataset
#!unzip building-sites-power-consumption-dataset.zip -d ./dataset

# Implementation

## Packages

In [3]:
# data elaboration functions
import pandas as pd
import string
import numpy as np

# datetime functions
import datetime as dt

# file management functions
import os
import sys
import opendatasets as od
import pickle
from pathlib import Path

# plot functions
import matplotlib.pyplot as plt
%matplotlib inline

# data science functions
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import joblib
from sklearn.metrics import mean_absolute_error

# configuration file
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from Configuration.config import cfg_path

# custom functions
from Code.Plotting.plots import Plots
from Code.Regressors.regressors import Regressors
from Code.Regressors.temperatures import Temperatures
from Code.Scoring.scoring import Scoring
from Code.Scoring.train_test import TrainTest
from Code.Scoring.train import Training
from Code.Scoring.forecast import Forecasting
from Code.Scoring.kpi import Kpi
from Code.Scoring.scoring import Scoring
from Code.Utils.utils import Utils


## Setup

In [4]:
#od.download("https://www.kaggle.com/arashnic/building-sites-power-consumption-dataset/download")
root = Path(os.getcwd()).parent
dataset_path = os.path.join(root, cfg_path.data_dir.input_path)

## Load Data


In [5]:
df_train_data = pd.read_csv(os.path.join(dataset_path, "power-laws-forecasting-energy-consumption-training-data.csv"), sep=";", parse_dates=["Timestamp"])
df_train_data = Utils.columns_camel_to_snake(df_train_data)
print("Train columns: ", list(df_train_data.columns))
print("NaNs:", df_train_data.isna().sum().values.sum())

df_metadata = pd.read_csv(os.path.join(dataset_path, "power-laws-forecasting-energy-consumption-metadata.csv"), sep=";")
df_metadata = Utils.columns_camel_to_snake(df_metadata)
print("Metadata columns: ", list(df_metadata.columns))
print("NaNs:", df_metadata.isna().sum().values.sum())

df_weather = pd.read_csv(os.path.join(dataset_path, "power-laws-forecasting-energy-consumption-weather.csv"), sep=";", parse_dates=["Timestamp"])
df_weather = Utils.columns_camel_to_snake(df_weather)
print("Weather columns: ", list(df_weather.columns))
print("NaNs:", df_weather.isna().sum().values.sum())

Train columns:  ['obs_id', 'site_id', 'timestamp', 'forecast_id', 'value']
NaNs: 86601
Metadata columns:  ['site_id', 'surface', 'sampling', 'base_temperature', 'monday_is_day_off', 'tuesday_is_day_off', 'wednesday_is_day_off', 'thursday_is_day_off', 'friday_is_day_off', 'saturday_is_day_off', 'sunday_is_day_off']
NaNs: 0
Weather columns:  ['timestamp', 'temperature', 'distance', 'site_id']
NaNs: 0


# Data Preparation


## Parameter setup

In [6]:
id = 'site_id'
list_unique_id = ['site_id', 'timestamp']
list_temp = ['temp']
y = 'value'

#### Setting forecast end date

In [7]:
# Make sure to have all regressors available until forecast_end_date (temperatures, etc)
forecast_end_date = '2022-12-31'

## Plotting y series

In [8]:
# Print available ids and choose which one to plot 
print(list(df_train_data[id].unique()))

[42, 43, 40, 41, 63, 66, 67, 68, 64, 65, 69, 70, 72, 73, 115, 119, 116, 117, 118, 59, 60, 62, 61, 112, 109, 110, 111, 8, 9, 25, 26, 46, 47, 48, 49, 106, 107, 100, 101, 102, 105, 50, 51, 52, 53, 108, 14, 10, 11, 12, 13, 22, 23, 57, 54, 58, 93, 94, 96, 98, 99, 92, 44, 45, 88, 87, 89, 90, 84, 85, 86, 78, 83, 228, 222, 223, 224, 225, 226, 227, 215, 229, 230, 273, 276, 271, 274, 275, 272, 74, 77, 76, 75, 218, 219, 221, 233, 234, 269, 270, 235, 236, 237, 216, 217, 204, 209, 261, 262, 263, 264, 265, 266, 267, 268, 232, 212, 213, 205, 206, 207, 208, 210, 211, 260, 259, 255, 256, 257, 231, 252, 254, 250, 251, 253, 238, 239, 240, 241, 243, 244, 245, 246, 247, 248, 249, 123, 124, 125, 126, 121, 122, 120, 127, 128, 129, 200, 201, 199, 141, 140, 135, 136, 139, 202, 203, 151, 149, 152, 146, 148, 150, 190, 191, 19, 167, 162, 173, 171, 172, 169, 170, 163, 164, 165, 143, 145, 142, 197, 194, 195, 196, 192, 193, 160, 161, 153, 154, 155, 156, 157, 158, 159, 198, 175, 174, 183, 184, 185, 186, 189, 181, 182

In [9]:
list_ids_to_plot = [49, 12, 63, 44]

In [10]:
count = 1
for i in list_ids_to_plot:
    print('Plotting id:', i, 'as', count, 'of', len(list_ids_to_plot))
    plot = Plots.sliding_line_plot(df_train_data, y, id, i, chart_title="")
    plot.write_html(os.path.join(root, cfg_path.data_dir.plot_path, id + '_' + str(i) + ".html"))
    count = count + 1 

Plotting id: 49 as 1 of 4
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 49
Plotting id: 12 as 2 of 4
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 12
Plotting id: 63 as 3 of 4
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 63
Plotting id: 44 as 4 of 4
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 44


## Dealing with NAs and aggregating at a chosen frequency

Create a full time sequence on a chosen frequency and aggregate

#### Consumption data (y)

In [11]:
# df_train_data
df_train_data.head()
date_var = Utils.find_date(df_train_data)
print('List ids:', list(df_train_data[id].unique()))
len(list(df_train_data[id].unique()))

find_date, date_col found: ['timestamp']
List ids: [42, 43, 40, 41, 63, 66, 67, 68, 64, 65, 69, 70, 72, 73, 115, 119, 116, 117, 118, 59, 60, 62, 61, 112, 109, 110, 111, 8, 9, 25, 26, 46, 47, 48, 49, 106, 107, 100, 101, 102, 105, 50, 51, 52, 53, 108, 14, 10, 11, 12, 13, 22, 23, 57, 54, 58, 93, 94, 96, 98, 99, 92, 44, 45, 88, 87, 89, 90, 84, 85, 86, 78, 83, 228, 222, 223, 224, 225, 226, 227, 215, 229, 230, 273, 276, 271, 274, 275, 272, 74, 77, 76, 75, 218, 219, 221, 233, 234, 269, 270, 235, 236, 237, 216, 217, 204, 209, 261, 262, 263, 264, 265, 266, 267, 268, 232, 212, 213, 205, 206, 207, 208, 210, 211, 260, 259, 255, 256, 257, 231, 252, 254, 250, 251, 253, 238, 239, 240, 241, 243, 244, 245, 246, 247, 248, 249, 123, 124, 125, 126, 121, 122, 120, 127, 128, 129, 200, 201, 199, 141, 140, 135, 136, 139, 202, 203, 151, 149, 152, 146, 148, 150, 190, 191, 19, 167, 162, 173, 171, 172, 169, 170, 163, 164, 165, 143, 145, 142, 197, 194, 195, 196, 192, 193, 160, 161, 153, 154, 155, 156, 157, 158, 15

267

In [12]:
# Resampling function aggregates data in a dataframe with a chosen function, that can vary depending on the variable
# i.e. temperatures when aggregated should be averaged, consumption should be summed, dummy variables should be pick as 'first'

df_train_data[date_var].apply(lambda x: x.tz_localize(None))
sampling = dt.timedelta(days=1)
dict_grouping = {'obs_id': 'first', 'forecast_id': 'first', 'value': 'sum'}
df_resampled = Utils.resample_data(df_train_data, id, date_var, sampling, dict_grouping)
print('List ids after resampling:', list(df_resampled[id].unique()))

resample_data: variable obs_id
resample_data: variable obs_id completed
resample_data: variable forecast_id
resample_data: variable forecast_id completed
resample_data: variable value
resample_data: variable value completed
                       timestamp     obs_id  site_id  forecast_id  \
0      2015-11-02 00:00:00+00:00  3187477.0       42       1080.0   
1      2015-11-03 00:00:00+00:00  1309542.0       42       1080.0   
2      2015-11-04 00:00:00+00:00  6998589.0       42       1080.0   
3      2015-11-05 00:00:00+00:00  5708558.0       42       1080.0   
4      2015-11-06 00:00:00+00:00  6931527.0       42       1080.0   
...                          ...        ...      ...          ...   
188662 2015-02-12 00:00:00+00:00   954528.0       34        981.0   
188663 2015-02-13 00:00:00+00:00  6319073.0       34        981.0   
188664 2015-02-14 00:00:00+00:00   849485.0       34        981.0   
188665 2015-02-15 00:00:00+00:00   141866.0       34        981.0   
188666 2015-02-16

In [13]:
# Adding a full time sequence
df_train_data = Utils.add_seq(df_resampled, date_var, serie = id, freq = sampling, end_date=forecast_end_date, start_date='')

Adding sequence to serie 42 as 1 of 267
Adding sequence to serie 43 as 2 of 267
Adding sequence to serie 40 as 3 of 267
Adding sequence to serie 41 as 4 of 267
Adding sequence to serie 63 as 5 of 267
Adding sequence to serie 66 as 6 of 267
Adding sequence to serie 67 as 7 of 267
Adding sequence to serie 68 as 8 of 267
Adding sequence to serie 64 as 9 of 267
Adding sequence to serie 65 as 10 of 267
Adding sequence to serie 69 as 11 of 267
Adding sequence to serie 70 as 12 of 267
Adding sequence to serie 72 as 13 of 267
Adding sequence to serie 73 as 14 of 267
Adding sequence to serie 115 as 15 of 267
Adding sequence to serie 119 as 16 of 267
Adding sequence to serie 116 as 17 of 267
Adding sequence to serie 117 as 18 of 267
Adding sequence to serie 118 as 19 of 267
Adding sequence to serie 59 as 20 of 267
Adding sequence to serie 60 as 21 of 267
Adding sequence to serie 62 as 22 of 267
Adding sequence to serie 61 as 23 of 267
Adding sequence to serie 112 as 24 of 267
Adding sequence to 

In [14]:
# This function count the number of obs you should have if you had a full time sequence
Utils.check_length_time_serie(df_train_data, date_var, index = id).head()

Expected length of sequence is OK 
      site_id   count  expected_obs
0          1  2617.0        2617.0
1          2  2617.0        2617.0
2          3  2617.0        2617.0
3          5  2617.0        2617.0
4          6  2617.0        2617.0
..       ...     ...           ...
262      301  2617.0        2617.0
263      302  2617.0        2617.0
264      303  2617.0        2617.0
265      304  2617.0        2617.0
266      305  2617.0        2617.0

[267 rows x 3 columns]


,site_id,count,min,max,td,freq,expected_obs,mismatch
0,1,2617.0,2015-11-02,2022-12-31,2616 days,D,2617.0,0
1,2,2617.0,2015-11-02,2022-12-31,2616 days,D,2617.0,0
2,3,2617.0,2015-11-02,2022-12-31,2616 days,D,2617.0,0
3,5,2617.0,2015-11-02,2022-12-31,2616 days,D,2617.0,0
4,6,2617.0,2015-11-02,2022-12-31,2616 days,D,2617.0,0


In [15]:
df_train_data.head()

,site_id,timestamp,obs_id,forecast_id,value
0,42,2015-11-02,3187477.0,1080.0,6.370880e+05
1,42,2015-11-03,1309542.0,1080.0,1.006306e+06
2,42,2015-11-04,6998589.0,1080.0,1.008557e+06
3,42,2015-11-05,5708558.0,1080.0,1.017008e+06
4,42,2015-11-06,6931527.0,1080.0,1.028142e+06


In [16]:
print('List ids after resampling and adding full time sequence:', list(df_train_data[id].unique()))

List ids after resampling and adding full time sequence: [42, 43, 40, 41, 63, 66, 67, 68, 64, 65, 69, 70, 72, 73, 115, 119, 116, 117, 118, 59, 60, 62, 61, 112, 109, 110, 111, 8, 9, 25, 26, 46, 47, 48, 49, 106, 107, 100, 101, 102, 105, 50, 51, 52, 53, 108, 14, 10, 11, 12, 13, 22, 23, 57, 54, 58, 93, 94, 96, 98, 99, 92, 44, 45, 88, 87, 89, 90, 84, 85, 86, 78, 83, 228, 222, 223, 224, 225, 226, 227, 215, 229, 230, 273, 276, 271, 274, 275, 272, 74, 77, 76, 75, 218, 219, 221, 233, 234, 269, 270, 235, 236, 237, 216, 217, 204, 209, 261, 262, 263, 264, 265, 266, 267, 268, 232, 212, 213, 205, 206, 207, 208, 210, 211, 260, 259, 255, 256, 257, 231, 252, 254, 250, 251, 253, 238, 239, 240, 241, 243, 244, 245, 246, 247, 248, 249, 123, 124, 125, 126, 121, 122, 120, 127, 128, 129, 200, 201, 199, 141, 140, 135, 136, 139, 202, 203, 151, 149, 152, 146, 148, 150, 190, 191, 19, 167, 162, 173, 171, 172, 169, 170, 163, 164, 165, 143, 145, 142, 197, 194, 195, 196, 192, 193, 160, 161, 153, 154, 155, 156, 157, 1

#### Weather data

In [17]:
# df_weather
date_var = Utils.find_date(df_weather)

find_date, date_col found: ['timestamp']


In [18]:
# Resampling function aggregates data in a dataframe with a chosen function, that can vary depending on the variable
# i.e. temperatures when aggregated should be averaged, consumption should be summed, dummy variables should be pick as 'first'

df_weather[date_var] = df_weather[date_var].apply(lambda x: x.tz_localize(None))
sampling = dt.timedelta(days=1)
dict_grouping = {'temperature': 'mean', 'distance': 'mean'}
df_resampled = Utils.resample_data(df_weather, id, date_var, sampling, dict_grouping)

resample_data: variable temperature
resample_data: variable temperature completed
resample_data: variable distance
resample_data: variable distance completed
       timestamp  temperature  site_id   distance
0     2015-01-01    -0.038168       51  28.196452
1     2015-01-02     1.806015       51  28.194644
2     2015-01-03     3.314179       51  28.102067
3     2015-01-04     4.661654       51  28.089319
4     2015-01-05     5.380741       51  28.615182
...          ...          ...      ...        ...
61557 2017-12-09          NaN       13        NaN
61558 2017-12-10     4.650000       13  25.252279
61559 2017-12-11     9.250000       13  24.200407
61560 2017-12-12     7.650000       13  25.252279
61561 2017-12-13     5.000000       13  24.200407

[61562 rows x 4 columns]


In [19]:
# Adding a full time sequence
df_weather = Utils.add_seq(df_resampled, date_var, serie = id, freq = sampling, end_date=forecast_end_date, start_date='')

Adding sequence to serie 51 as 1 of 49
Adding sequence to serie 50 as 2 of 49
Adding sequence to serie 56 as 3 of 49
Adding sequence to serie 57 as 4 of 49
Adding sequence to serie 20 as 5 of 49
Adding sequence to serie 21 as 6 of 49
Adding sequence to serie 19 as 7 of 49
Adding sequence to serie 22 as 8 of 49
Adding sequence to serie 25 as 9 of 49
Adding sequence to serie 26 as 10 of 49
Adding sequence to serie 29 as 11 of 49
Adding sequence to serie 28 as 12 of 49
Adding sequence to serie 27 as 13 of 49
Adding sequence to serie 30 as 14 of 49
Adding sequence to serie 55 as 15 of 49
Adding sequence to serie 52 as 16 of 49
Adding sequence to serie 24 as 17 of 49
Adding sequence to serie 23 as 18 of 49
Adding sequence to serie 36 as 19 of 49
Adding sequence to serie 35 as 20 of 49
Adding sequence to serie 48 as 21 of 49
Adding sequence to serie 47 as 22 of 49
Adding sequence to serie 38 as 23 of 49
Adding sequence to serie 37 as 24 of 49
Adding sequence to serie 49 as 25 of 49
Adding se

In [20]:
# This function count the number of obs you should have if you had a full time sequence
Utils.check_length_time_serie(df_weather, date_var, index = id).head()

Expected length of sequence is OK 
     site_id   count  expected_obs
0         2  2922.0        2922.0
1         3  2922.0        2922.0
2         4  2922.0        2922.0
3         5  2922.0        2922.0
4         6  2922.0        2922.0
5         7  2922.0        2922.0
6         8  2922.0        2922.0
7         9  2922.0        2922.0
8        10  2922.0        2922.0
9        12  2922.0        2922.0
10       13  2922.0        2922.0
11       16  2922.0        2922.0
12       17  2922.0        2922.0
13       18  2922.0        2922.0
14       19  2922.0        2922.0
15       20  2922.0        2922.0
16       21  2922.0        2922.0
17       22  2922.0        2922.0
18       23  2922.0        2922.0
19       24  2922.0        2922.0
20       25  2922.0        2922.0
21       26  2922.0        2922.0
22       27  2922.0        2922.0
23       28  2922.0        2922.0
24       29  2922.0        2922.0
25       30  2922.0        2922.0
26       32  2922.0        2922.0
27       33 

,site_id,count,min,max,td,freq,expected_obs,mismatch
0,2,2922.0,2015-01-01,2022-12-31,2921 days,D,2922.0,0
1,3,2922.0,2015-01-01,2022-12-31,2921 days,D,2922.0,0
2,4,2922.0,2015-01-01,2022-12-31,2921 days,D,2922.0,0
3,5,2922.0,2015-01-01,2022-12-31,2921 days,D,2922.0,0
4,6,2922.0,2015-01-01,2022-12-31,2921 days,D,2922.0,0


In [21]:
# Adding ten-year averages temperature to fill data until forecast_end_date
freq = Utils.find_freq_in_dataframe(df_weather, date_var)
temperature_list = ['temperature']
df_weather_ten_year = Temperatures.ten_year(df_weather, id, date_var, freq, temperature_list, start_date ='', end_date=forecast_end_date)

Adding sequence to serie 51 as 1 of 49
Adding sequence to serie 50 as 2 of 49
Adding sequence to serie 56 as 3 of 49
Adding sequence to serie 57 as 4 of 49
Adding sequence to serie 20 as 5 of 49
Adding sequence to serie 21 as 6 of 49
Adding sequence to serie 19 as 7 of 49
Adding sequence to serie 22 as 8 of 49
Adding sequence to serie 25 as 9 of 49
Adding sequence to serie 26 as 10 of 49
Adding sequence to serie 29 as 11 of 49
Adding sequence to serie 28 as 12 of 49
Adding sequence to serie 27 as 13 of 49
Adding sequence to serie 30 as 14 of 49
Adding sequence to serie 55 as 15 of 49
Adding sequence to serie 52 as 16 of 49
Adding sequence to serie 24 as 17 of 49
Adding sequence to serie 23 as 18 of 49
Adding sequence to serie 36 as 19 of 49
Adding sequence to serie 35 as 20 of 49
Adding sequence to serie 48 as 21 of 49
Adding sequence to serie 47 as 22 of 49
Adding sequence to serie 38 as 23 of 49
Adding sequence to serie 37 as 24 of 49
Adding sequence to serie 49 as 25 of 49
Adding se

#### Additional data

In [22]:
# df_metadata 
# does not contain a time series, therefore it cannot be resampled and do not need a full time sequence to check for NAs
df_metadata.head()

,site_id,surface,sampling,base_temperature,monday_is_day_off,tuesday_is_day_off,wednesday_is_day_off,thursday_is_day_off,friday_is_day_off,saturday_is_day_off,sunday_is_day_off
0,207,7964.873347,30.0,18.0,False,False,False,False,False,True,True
1,7,15168.125971,30.0,18.0,False,False,False,False,False,True,True
2,74,424.340663,15.0,18.0,False,False,False,False,False,True,True
3,239,1164.822636,15.0,18.0,False,False,False,False,False,True,True
4,274,1468.246690,5.0,18.0,False,False,False,False,False,True,True


## Creating working dataset

In [23]:
# Site Ids available in all data dfs
common_site_ids = list(set(set(df_weather_ten_year[id].unique()) & set(df_train_data[id].unique())))

# Final df
df_final = df_train_data[df_train_data[id].isin(common_site_ids)].copy()

# Date
date_var = Utils.find_date(df_final)

find_date, date_col found: ['timestamp']


#### Count NAs in y by id

In [24]:
df_final.head()

,site_id,timestamp,obs_id,forecast_id,value
0,42,2015-11-02,3187477.0,1080.0,6.370880e+05
1,42,2015-11-03,1309542.0,1080.0,1.006306e+06
2,42,2015-11-04,6998589.0,1080.0,1.008557e+06
3,42,2015-11-05,5708558.0,1080.0,1.017008e+06
4,42,2015-11-06,6931527.0,1080.0,1.028142e+06


In [25]:
pivotna = pd.pivot_table(df_final[df_final[y].isna()], index=id, values = y, aggfunc='count').reset_index()
pivotna.rename(columns={y: y + '_count_NA'})
pivotna

,site_id,value
0,2,0
1,3,0
2,5,0
3,6,0
4,7,0
5,8,0
6,9,0
7,10,0
8,12,0
9,13,0


### Adding regressors to final dataframe

#### Holidays

If you don't have specific holiday dataset, you can use the following general function by country that uses the holiday python package and adds to your dataframe a columns with a holiday dummy variable (0/1):

    df_final = Regressors.add_holidays_by_country(df_final, date_var, country = 'France')

In [26]:
df_final = Regressors.add_holidays_by_country(df_final, date_var, country='India')
print('Min date:', df_final[date_var].min())
print('Max date:', df_final[date_var].max())

Min date: 2015-11-02 00:00:00
Max date: 2022-12-31 00:00:00


#### Site leaves

In [27]:
dict_days_off = {'friday_is_day_off': 5, 'saturday_is_day_off': 6, 'sunday_is_day_off': 7}
df_final = Regressors.merge_additional_days_off(df_final, df_metadata, id, dict_days_off)
print('Min date:', df_final[date_var].min())
print('Max date:', df_final[date_var].max())

find_date, date_col found: ['timestamp']
Min date: 2015-11-02 00:00:00
Max date: 2022-12-31 00:00:00


#### Additional metadata

In [28]:
df_final = pd.merge(df_final, df_metadata[[id, "surface", "base_temperature"]], how="left", on=[id], validate = 'm:1')
print('Min date:', df_final[date_var].min())
print('Max date:', df_final[date_var].max())

Min date: 2015-11-02 00:00:00
Max date: 2022-12-31 00:00:00


#### Other calendar variables

In [29]:
df_final = Regressors.add_weekdays(df_final, date_var)
df_final = Regressors.add_months(df_final, date_var)
print('Min date:', df_final[date_var].min())
print('Max date:', df_final[date_var].max())

Min date: 2015-11-02 00:00:00
Max date: 2022-12-31 00:00:00


#### Weather

In [30]:
df_final = Regressors.merge_weather(df_final, df_weather_ten_year, date_var, id)
print('Min date:', df_final[date_var].min())
print('Max date:', df_final[date_var].max())

find_date, date_col found: ['timestamp']
find_date, date_col found: ['timestamp']
Min date: 2015-11-02 00:00:00
Max date: 2022-12-31 00:00:00


In [31]:
df_final = Regressors.calculate_degree_days(df_final, base_temperature = "base_temperature", temperature = "temperature")
print('Min date:', df_final[date_var].min())
print('Max date:', df_final[date_var].max())

Min date: 2015-11-02 00:00:00
Max date: 2022-12-31 00:00:00


#### Remove duplicates

In [32]:
df_final = df_final.drop_duplicates()
print('List ids in df_final after removing duplicates:', list(df_final[id].unique()))
assert df_final[df_final.duplicated()].count().sum() == 0, "y should not contain duplicates"
print('Min date:', df_final[date_var].min())
print('Max date:', df_final[date_var].max())

List ids in df_final after removing duplicates: [2, 3, 5, 6, 7, 8, 9, 10, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 29, 32, 33, 34, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 57]
Min date: 2015-11-02 00:00:00
Max date: 2022-12-31 00:00:00


#### Check regressor availability

In [33]:
df_final.columns

Index(['site_id', 'timestamp', 'obs_id', 'forecast_id', 'value', 'holidays',
       'day_off', 'surface', 'base_temperature', 'wd_mon', 'wd_tue', 'wd_wed',
       'wd_thu', 'wd_fri', 'wd_sat', 'wd_sun', 'month_01', 'month_02',
       'month_03', 'month_04', 'month_05', 'month_06', 'month_07', 'month_08',
       'month_09', 'month_10', 'month_11', 'month_12', 'temperature',
       'distance', 'months_days', 'temperature_ten_year', 'temperature_asis',
       'DDC_temperature', 'DDH_temperature'],
      dtype='object')

In [39]:
# Temperatures have been filled, only temperature asis that is the composition between the actual temperature and ten year averages
regressors_list = [ 'holidays',
       'day_off', 'surface', 'base_temperature', 'wd_mon', 'wd_tue', 'wd_wed',
       'wd_thu', 'wd_fri', 'wd_sat', 'wd_sun', 'month_01', 'month_02',
       'month_03', 'month_04', 'month_05', 'month_06', 'month_07', 'month_08',
       'month_09', 'month_10', 'month_11', 'month_12', 'temperature',
       'distance', 'months_days', 'temperature_ten_year', 'temperature_asis',
       'DDC_temperature', 'DDH_temperature']

try:
       Utils.check_regressors_availability(df_final, date_var, regressors_list, forecast_end_date)
except:
       Utils.remove_regressors_with_nan(df_final, date_var, regressors_list, forecast_end_date)

Regressor holidays has all needed values
Regressor day_off has all needed values
Regressor surface has all needed values
Regressor base_temperature has all needed values
Regressor wd_mon has all needed values
Regressor wd_tue has all needed values
Regressor wd_wed has all needed values
Regressor wd_thu has all needed values
Regressor wd_fri has all needed values
Regressor wd_sat has all needed values
Regressor wd_sun has all needed values
Regressor month_01 has all needed values
Regressor month_02 has all needed values
Regressor month_03 has all needed values
Regressor month_04 has all needed values
Regressor month_05 has all needed values
Regressor month_06 has all needed values
Regressor month_07 has all needed values
Regressor month_08 has all needed values
Regressor month_09 has all needed values
Regressor month_10 has all needed values
Regressor month_11 has all needed values
Regressor month_12 has all needed values
Latest filled available date for regressor temperature is 2017-12

# Saving

In [40]:
df_final.to_pickle(os.path.join(root, cfg_path.data_dir.output_path, 'df_final.pkl'))

In [41]:
print('Min date:', df_final[date_var].min())
print('Max date:', df_final[date_var].max())
df_final.head()


Min date: 2015-11-02 00:00:00
Max date: 2022-12-31 00:00:00


,site_id,timestamp,obs_id,forecast_id,value,holidays,day_off,surface,base_temperature,wd_mon,...,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12,months_days,temperature_asis
0,2,2015-11-02,7390465.0,26.0,1.492533e+06,0,0,6098.278376,18.0,1,...,0,0,0,0,0,0,1,0,11/02,13.247917
1,3,2015-11-02,NaN,NaN,NaN,0,0,10556.293605,18.0,1,...,0,0,0,0,0,0,1,0,11/02,13.252083
2,5,2015-11-02,NaN,NaN,NaN,0,0,12541.181277,18.0,1,...,0,0,0,0,0,0,1,0,11/02,13.241667
3,6,2015-11-02,1413383.0,129.0,8.541479e+05,0,0,9150.195373,18.0,1,...,0,0,0,0,0,0,1,0,11/02,13.243750
4,7,2015-11-02,NaN,NaN,NaN,0,0,15168.125971,18.0,1,...,0,0,0,0,0,0,1,0,11/02,13.245833
